In [1]:
import pandas as pd
import os
import sklearn
import shap
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_validate,RandomizedSearchCV, StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
from trainMLmodel import surfaceDesignTopAndBottom,selectDescriptors,combineRankAndDescriptorsClassyfication,explainer, combineRankAndDescriptorsClassificationbotisFLAT, combineRankAndDescriptorsClassyfication_Danilingus
import numpy as np
# Define the location of the csv files imageOutliersRemoved and Ranking
#pathToImageObject=os.getcwd()+"/DataAnalysis/"
pathToImageObject="/bme001.mnt/home/20212358"
locationOfImageObject1="cleaned_rawImagedataFI_final.csv"
locationOfImageObject2="DataAnalysis_withJan/Screen_Mean_Count_CellBody_new.csv"
# Define the location of the annotation files
pathToTopoUnitImages="/FeatureImages/FeatureImages/"
allTopoDescriptors=pd.read_csv("allTopoDescriptors.csv")
# check if file exists
if os.path.isfile(locationOfImageObject1):
    imagedataOutliersRemoved=pd.read_csv(locationOfImageObject1,low_memory=False)
    if os.path.isfile(locationOfImageObject2):
        imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)
    else:
        print("File not found, check if you finished ranking the features")
else:
    print('File not find, check if you removed the outliers')


In [29]:
import pandas as pd

# Suppose your full DataFrame is 'df'
target_col = "Mean_CellBody_new_AreaShape_Solidity"
#Mean_CellBody_new_Intensity_IntegratedIntensity_CorrDNA
#target_col2 = "Mean_CellBody_new_Intensity_IntegratedIntensity_CorrCellBody"
# 1. Create a mask to ensure we only handle numeric columns
# (corrwith requires numeric data, so we filter out non-numeric columns if any)
numeric_cols = imagedataOutliersRemoved.select_dtypes(include=[float, int]).columns

# 2. Compute correlations for all numeric columns against your target column
pearson_corr = imagedataOutliersRemoved[numeric_cols].corrwith(imagedataOutliersRemoved[target_col], method='pearson')

# 3. Compute Spearman correlations for all numeric columns against your target column
spearman_corr = imagedataOutliersRemoved[numeric_cols].corrwith(imagedataOutliersRemoved[target_col], method='spearman')

# 4. Combine results into a single DataFrame for convenience
corr_results = pd.DataFrame({
    'Pearson': pearson_corr,
    'Spearman': spearman_corr
})

# 5. Optionally, sort by absolute Pearson or Spearman correlation
corr_results = corr_results.reindex(
    corr_results['Pearson'].abs().sort_values(ascending=False).index
)

# 6. Display or save the correlation summary
print(corr_results)


/sw/rl8/zen/app/SciPy-bundle/2022.05-foss-2022a/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


                                                     Pearson  Spearman
Mean_CellBody_new_AreaShape_Solidity                1.000000  1.000000
Median_CellBody_new_AreaShape_Solidity              0.941101  0.937726
Mean_CellBody_new_AreaShape_FormFactor              0.921064  0.921487
Mean_CellBody_new_AreaShape_Extent                  0.916577  0.910485
Mean_CellBody_new_AreaShape_Compactness            -0.907294 -0.925034
...                                                      ...       ...
StDev_Nephrin_Location_CenterMassIntensity_Z_Co...       NaN       NaN
StDev_Nephrin_Location_Center_Z                          NaN       NaN
StDev_Nephrin_Location_MaxIntensity_Z_CorrCellBody       NaN       NaN
StDev_Nephrin_Location_MaxIntensity_Z_CorrDNA            NaN       NaN
StDev_Nephrin_Location_MaxIntensity_Z_CorrTarget         NaN       NaN

[5551 rows x 2 columns]


In [31]:
corr_results.to_csv("Correlations with CellBody solidity.csv", index=True)

In [9]:
# 1. Row count
num_rows = len(imagedataOutliersRemoved)

# 2. Column count excluding certain words
excluded_substrings = [
    "metadata", "Metadata", "FileName", "Length", "Number", "Height", "ImageNumber"
]
cols_to_keep = [
    col for col in imagedataOutliersRemoved.columns 
    if not any(ex_sub in col for ex_sub in excluded_substrings)
]
num_excluded_columns = len(cols_to_keep)

print("Number of rows:", num_rows)
print("Number of columns (excluding certain words):", num_excluded_columns)


Number of rows: 31920
Number of columns (excluding certain words): 5490


In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Example: if you've read your data into a pandas DataFrame called 'df'
# Make sure the column names below match exactly those in your DataFrame
columns_of_interest = [
    "Mean_CellBody_new_AreaShape_Solidity",
    "Mean_CellBody_new_RadialDistribution_FracAtD_CorrCellBody_4of4"
]

# Create a smaller DataFrame for just the columns you care about
df_subset = imagedataOutliersRemoved[columns_of_interest].dropna()

# 1. Compute the correlation matrix (using Pearson by default)
corr_matrix = df_subset.corr(method='pearson')
print("Correlation matrix (Pearson):\n", corr_matrix)

# 2. You can also try Spearman’s rank correlation if non-linear or non-normal data is suspected
corr_matrix_spearman = df_subset.corr(method='spearman')
print("\nCorrelation matrix (Spearman):\n", corr_matrix_spearman)

# 3. Visualize correlations with a heatmap
sns.heatmap(corr_matrix, annot=True, cmap="YlGnBu", vmin=-1, vmax=1)
plt.title("Correlation Heatmap (Pearson)")
plt.show()

sns.heatmap(corr_matrix_spearman, annot=True, cmap="YlGnBu", vmin=-1, vmax=1)
plt.title("Correlation Heatmap (Spearman)")
plt.show()

# 4. Example scatter plot (to visually inspect the relationship)
x_col = "Mean_CellBody_new_AreaShape_Solidity"
y_col = "Mean_CellBody_new_RadialDistribution_FracAtD_CorrCellBody_4of4"


plt.figure()
sns.scatterplot(data=df_subset, x=x_col, y=y_col)
plt.title(f"Scatter Plot: {x_col} vs {y_col}")
plt.show()


Correlation matrix (Pearson):
                                                     Mean_FilteredNuclei_new_Intensity_IntegratedIntensity_CorrDNA  \
Mean_FilteredNuclei_new_Intensity_IntegratedInt...                                           1.000000               
Mean_FilteredNuclei_new_AreaShape_MajorAxisLength                                            0.572153               

                                                    Mean_FilteredNuclei_new_AreaShape_MajorAxisLength  
Mean_FilteredNuclei_new_Intensity_IntegratedInt...                                           0.572153  
Mean_FilteredNuclei_new_AreaShape_MajorAxisLength                                            1.000000  

Correlation matrix (Spearman):
                                                     Mean_FilteredNuclei_new_Intensity_IntegratedIntensity_CorrDNA  \
Mean_FilteredNuclei_new_Intensity_IntegratedInt...                                           1.000000               
Mean_FilteredNuclei_new_AreaShape_Majo

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

# Set up the figure in centimeters (1 inch = 2.54 cm)
cm_to_inch = 1 / 2.54
fig_width = 8.5 * cm_to_inch
fig_height = 8.5 * cm_to_inch

columns_of_interest = [
    "Mean_CellBody_new_AreaShape_Solidity",
    "Mean_CellBody_new_RadialDistribution_FracAtD_CorrCellBody_4of4"
]

# Create a smaller DataFrame
df_subset = imagedataOutliersRemoved[columns_of_interest].dropna()

# Compute Pearson correlation
r, p = pearsonr(
    df_subset[columns_of_interest[0]],
    df_subset[columns_of_interest[1]]
)

# Create figure
plt.figure(figsize=(fig_width, fig_height))

# Scatter plot
sns.scatterplot(
    data=df_subset,
    x=columns_of_interest[0],
    y=columns_of_interest[1],
    s=20,
    alpha=0.7,
    edgecolor='none'
)

# Trend line
sns.regplot(
    data=df_subset,
    x=columns_of_interest[0],
    y=columns_of_interest[1],
    scatter=False,
    color='red',
    line_kws={"linewidth": 1, "linestyle": "--"}
)

# Axis labels – updated
plt.xlabel("Cell solidity (convexity)", fontsize=10, fontname="Arial")
plt.ylabel("Fraction of phalloidin in outermost ring (4/4)", fontsize=10, fontname="Arial")

# Ticks
plt.xticks(fontsize=9, fontname="Arial")
plt.yticks(fontsize=9, fontname="Arial")

# Pearson correlation text
plt.text(
    0.95, 0.95,
    f"Pearson r = {r:.2f}",
    transform=plt.gca().transAxes,
    fontsize=12,
    fontname='Arial',
    verticalalignment='top',
    horizontalalignment='right'  # 👈 this does the trick
)


# Final touches
plt.grid(False)
plt.tight_layout()
plt.savefig("scatter_solidity_vs_radialfrac_8.5cm.png", dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()


In [21]:
print(df_subset.columns.tolist())


['Mean_FilteredNuclei_new_Intensity_IntegratedIntensity_CorrDNA', 'Mean_FilteredNuclei_new_AreaShape_MajorAxisLength']


In [8]:
imageDataRankedSurfaces

FeatureIdx  Screen_ADpvalue_Count_CellBody_new  \
0            710                                0.25   
1           1877                                0.25   
2           1448                                0.25   
3           1076                                0.25   
4           1054                                0.25   
...          ...                                 ...   
1829        1288                                0.25   
1830        1742                                0.25   
1831         836                                0.25   
1832        1292                                0.25   
1833        1625                                0.25   

      Screen_KSpvalue_Count_CellBody_new  Screen_SNR_Count_CellBody_new  \
0                               0.474026                       3.799227   
1                               0.930736                       5.293229   
2                               0.962704                       3.913095   
3                               0.962704                       3.102444   
4                               0.660140                       2.805779   
...                                  ...                            ...   
1829                            0.637363                       2.343262   
1830                            0.660140                       2.257996   
1831                            0.735664                       2.389780   
1832                            0.316239                       2.175057   
1833                            1.000000                       2.795554   

      Screen_Mean_Count_CellBody_new  Screen_SD_Count_CellBody_new  \
0                          47.833333                     12.590280   
1                          47.583333                      8.989472   
2                          46.785714                     11.956193   
3                          46.428571                     14.965161   
4                          46.125000                     16.439282   
...                              ...                           ...   
1829                       24.571429                     10.485993   
1830                       24.466667                     10.835568   
1831                       24.000000                     10.042766   
1832                       23.384615                     10.751267   
1833                       19.833333                      7.094599   

      Screen_Median_Count_CellBody_new    Rank  
0                                 49.5     1.0  
1                                 47.5     2.0  
2                                 49.5     3.0  
3                                 53.5     4.0  
4                                 53.0     5.0  
...                                ...     ...  
1829                              23.0  1830.0  
1830                              24.0  1831.0  
1831                              21.0  1832.0  
1832                              22.0  1833.0  
1833                              18.5  1834.0  

[1834 rows x 8 columns]

In [5]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Target column
target_col = "Mean_FilteredNuclei_new_Intensity_IntegratedIntensity_CorrDNA"

# Filter only numeric columns (excluding the target)
numeric_cols = imagedataOutliersRemoved.select_dtypes(include=[float, int]).columns.drop(target_col)

# Prepare lists to store results
pearson_corrs = []
pearson_ps = []
spearman_corrs = []
spearman_ps = []
n_values = []

# Loop through each numeric column and calculate correlations with target_col
for col in numeric_cols:
    # Drop NA pairs
    paired_data = imagedataOutliersRemoved[[col, target_col]].dropna()
    x = paired_data[col]
    y = paired_data[target_col]
    
    if len(x) > 1:  # Ensure we have enough data
        # Pearson
        r_pearson, p_pearson = pearsonr(x, y)
        # Spearman
        r_spearman, p_spearman = spearmanr(x, y)
        
        pearson_corrs.append(r_pearson)
        pearson_ps.append(p_pearson)
        spearman_corrs.append(r_spearman)
        spearman_ps.append(p_spearman)
        n_values.append(len(x))
    else:
        pearson_corrs.append(None)
        pearson_ps.append(None)
        spearman_corrs.append(None)
        spearman_ps.append(None)
        n_values.append(len(x))

# Create results DataFrame
corr_results = pd.DataFrame({
    'Feature': numeric_cols,
    'n': n_values,
    'Pearson_r': pearson_corrs,
    'Pearson_p': pearson_ps,
    'Spearman_r': spearman_corrs,
    'Spearman_p': spearman_ps
})

# Sort by absolute Pearson_r (or Spearman_r if preferred)
corr_results = corr_results.set_index('Feature')
corr_results = corr_results.reindex(corr_results['Pearson_r'].abs().sort_values(ascending=False).index)

# Display
print(corr_results)


/sw/rl8/zen/app/SciPy-bundle/2022.05-foss-2022a/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/sw/rl8/zen/app/SciPy-bundle/2022.05-foss-2022a/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/sw/rl8/zen/app/SciPy-bundle/2022.05-foss-2022a/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/sw/rl8/zen/app/SciPy-bundle/2022.05-foss-2022a/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.wa

                                                        n  Pearson_r  \
Feature                                                                
Mean_FilteredNuclei_target_Intensity_Integrated...  31920   0.985334   
Median_FilteredNuclei_new_Intensity_IntegratedI...  31920   0.930715   
Mean_FilteredNuclei_new_Intensity_IntegratedInt...  31920   0.921851   
Median_FilteredNuclei_target_Intensity_Integrat...  31920   0.916746   
Mean_FilteredNuclei_target_Intensity_Integrated...  31920   0.916218   
...                                                   ...        ...   
StDev_Nephrin_Location_CenterMassIntensity_Z_Co...  31920        NaN   
StDev_Nephrin_Location_Center_Z                     31920        NaN   
StDev_Nephrin_Location_MaxIntensity_Z_CorrCellBody  31920        NaN   
StDev_Nephrin_Location_MaxIntensity_Z_CorrDNA       31920        NaN   
StDev_Nephrin_Location_MaxIntensity_Z_CorrTarget    31920        NaN   

                                                    Pearson_p  

In [6]:
corr_results.to_csv("Correlations with dapi intensity_with_p-values.csv", index=True)